# Advanced Python - Building Scalable Applications

### Module 5

#### Sharing and Exchanging data between processes
 - Streaming data using ```Pipe``` and ```Queue```
 - Sharing counters and buffers using ```Value``` and ```Array```
 - Sharing python lists and dictionaries using ```Manager```
 - Creating and managing shared memory using ```multiprocessing.shared_memory``` features

#### Profiling and Debugging Techniques in Python
 - Using `sys.getsizeof()`, `sys.getrefcount()`, `system.getswitchinterval()`
 - Using `cProfile` and `timeit` modules
 - Using `line_profiler` and `Memray`
 - Using `inspect` and `pdb`
 - Using the `logging` module

In [ ]:
from queue import Queue

q = Queue(10)


In [1]:
from multiprocessing import Queue

Queue?

Signature: Queue(maxsize=0)
Docstring: Returns a queue object
File:      /opt/anaconda3/lib/python3.12/multiprocessing/context.py
Type:      method